In [ ]:
# default_exp retrieve_specific

In [ ]:
# export
import requests
import base64
import json
import pandas as pd
import boto3

from datetime import date, timedelta

In [ ]:
#hide
from nbdev.showdoc import *

# retrieval from net

> Pull all tracks from the playlist to be used as the net

In [ ]:
# export
def cred():
    secret_name = "spotify_35"
    region_name = "us-east-2"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    client_id = json.loads(get_secret_value_response['SecretString'])['spot_clientID']
    client_secret = json.loads(get_secret_value_response['SecretString'])['spot_clientSECRET']
    access_token = json.loads(get_secret_value_response['SecretString'])['spot_ACC']
    refresh_token = json.loads(get_secret_value_response['SecretString'])['spot_REF']

    return client_id, client_secret, access_token, refresh_token

In [ ]:
# export
pref = {
'tepid': '3ubgXaHeBn1CWLUZPXvqkj'
}

In [ ]:
# export
def get_tracks(p_id, access_token, refresh_token, client_id, client_secret):

    df_tracks = pd.DataFrame()

    curr_len = 0
    offset = 0

    while len(df_tracks) % 100 == 0:
        

        track_url = f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=100&offset={offset}'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        r_track = requests.get(track_url, headers=headers)

        if r_track.status_code == 401:

            TOKEN_URL = 'https://accounts.spotify.com/api/token'

            message = client_id + ':' + client_secret
            messageBytes = message.encode('ascii')
            base64Bytes = base64.b64encode(messageBytes)
            base64Message = base64Bytes.decode('ascii')
        
            headers = {
            'Authorization': 'Basic ' + base64Message,
            'Content-Type': 'application/x-www-form-urlencoded'
            }

            pars_refresh = {
                'grant_type': 'refresh_token',
                'refresh_token': refresh_token,
                'redirect_uri': 'http://localhost:8888/callback',
            }

            r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
            access_token = r_refresh.json()['access_token']

            headers = {
            'Authorization': f'Bearer {access_token}'
                    }

            r_track = requests.get(track_url, headers=headers)

        track_ids = [t['track']['uri'] for t in r_track.json()['items']]
        track_names = [t['track']['name'] for t in r_track.json()['items']]
        track_added = [t['added_at'] for t in r_track.json()['items']]

        df_t = pd.DataFrame({
                'name': track_names,
                'id': track_ids,
                'playlist id': p_id

        }, 
        index=pd.to_datetime(track_added)  
        )

        # if df_t['id']
        df_tracks = df_tracks.append(df_t)
        if curr_len == len(df_tracks):
            break
        else:
            curr_len = len(df_tracks)

        offset += 100

    return df_tracks

In [ ]:
if __name__ == '__main__':
    c_id, c_secret, a_token, r_token = cred()
    d_tracks = get_tracks('3ubgXaHeBn1CWLUZPXvqkj', a_token, r_token, c_id, c_secret)

In [ ]:
d_tracks = d_tracks.reset_index()
d_tracks = d_tracks.sort_values('index')

In [ ]:
today = date.today()
today = pd.to_datetime(today, utc=True)

In [ ]:
d_tracks['diff'] = d_tracks['index'].apply(lambda x: today-x)

In [ ]:
period = timedelta(days=30)

In [ ]:
d_tracks = d_tracks[d_tracks['diff'].apply(lambda x: x <= period)]
d_tracks = d_tracks.sort_values('diff')

In [ ]:
d_tracks

,index,name,id,playlist id,diff
411,2022-01-11 19:54:01+00:00,Endora,spotify:track:0JULjPLL0XCQOrNKcJlAq5,3ubgXaHeBn1CWLUZPXvqkj,-1 days +04:05:59
410,2022-01-11 15:54:28+00:00,Gronlandic Edit,spotify:track:3X8gOyVXP93Tslnz9Bxw24,3ubgXaHeBn1CWLUZPXvqkj,-1 days +08:05:32
409,2022-01-11 15:47:55+00:00,Wraith Pinned to the Mist and Other Games,spotify:track:24HPkbkXJsIFC4eyg63zgQ,3ubgXaHeBn1CWLUZPXvqkj,-1 days +08:12:05
408,2022-01-11 15:14:13+00:00,Believer - Marsh's Guatape Remix,spotify:track:1GHOLip5jcgRIBqd8QUx7N,3ubgXaHeBn1CWLUZPXvqkj,-1 days +08:45:47
407,2022-01-11 15:10:00+00:00,Burning (feat. Camden Cox) - Leftwing : Kody R...,spotify:track:5mtQYsaEApKNtjQWAR5nRQ,3ubgXaHeBn1CWLUZPXvqkj,-1 days +08:50:00
406,2022-01-06 13:38:59+00:00,PATTERNS,spotify:track:5BFmKRDd5AfV2mKvNSxyRG,3ubgXaHeBn1CWLUZPXvqkj,4 days 10:21:01
405,2022-01-06 13:13:47+00:00,I SEE IT COMING,spotify:track:4qDVck33icBULgnvP6WmPh,3ubgXaHeBn1CWLUZPXvqkj,4 days 10:46:13
404,2022-01-06 13:11:56+00:00,Strangers (with Mansionair),spotify:track:5nBqvq3hUl8n6HtKtwNphW,3ubgXaHeBn1CWLUZPXvqkj,4 days 10:48:04
403,2022-01-04 16:49:42+00:00,MENTHOL*,spotify:track:72rEOdEzgGc81Gos5dD0Je,3ubgXaHeBn1CWLUZPXvqkj,6 days 07:10:18
402,2022-01-04 16:26:12+00:00,What You Like,spotify:track:0h6ENY9kDXxfbbfnX3onPR,3ubgXaHeBn1CWLUZPXvqkj,6 days 07:33:48
